# Stand Anomalie
wie es scheint liegt das "problem" daran, wenn ein Kunde mehr Items im Warenkorb hat. Needs to be testet. Anomalie sorgt dafür, dass nach einer normalisierung es eine Reihe zu viel gibt. 

In [ ]:
import requests
import pandas as pd
from dotenv import load_dotenv 
import os 
from sql_functions import encrypt
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)


In [ ]:
load_dotenv()

In [ ]:
# Define the API endpoint, headers and used variables
url = 'https://www.wixapis.com/stores/v2/orders/query'
rows_offset = 0
counter = [1]
data = pd.DataFrame()
wix_orders = pd.DataFrame()
line_items = pd.DataFrame()
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json, text/plain, */*',
    'Authorization': os.getenv("wix_api_key"),
    'wix-site-id': os.getenv("statistik_side_id")  
}

for i in counter:
    # Define the query payload as a dictionary
    query_payload = {
        "query": {
            "paging": {
                "offset": rows_offset
            }
        }
    }

    # Make an HTTP POST request
    response = requests.post(url, json=query_payload, headers=headers)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        data = data.get("orders")

        # Normalize and concatenate the wanted data
        temp_line_items = pd.json_normalize(data, record_path= ["lineItems"], meta = ["number","id"], sep="_")
        temporary_df = pd.json_normalize(data, sep="_") 
        wix_orders = pd.concat([temporary_df,wix_orders])
        line_items = pd.concat([temp_line_items,line_items])

        # Set counter and offset
        rows_offset += 100
        counter.append(1)

    else:
        print(f"Request failed with status code {response.status_code}: {response.text}")
        print("Or end of request. Check wix_orders's content")
        break
    


In [ ]:
#Encrypt Mails, billing and buyer mails are not the same. Also the columns with Mails are not deleted yet! 
encrypt(wix_orders,"buyerInfo_email")
encrypt(wix_orders,"billingInfo_address_email")

In [ ]:
# Drop personal data, all shipment columns and other unnecessary columns
drop_list = ["lineItems", "activities", "buyerInfo_firstName", "buyerInfo_lastName", "billingInfo_address_fullName_firstName" , "billingInfo_address_fullName_lastName", "billingInfo_address_subdivision", "billingInfo_address_zipCode", "billingInfo_address_city" , "billingInfo_address_street_number" , "billingInfo_address_street_name" , "buyerInfo_phone" , "billingInfo_address_phone" , "billingInfo_address_addressLine1", "shippingInfo_shippingRegion", "shippingInfo_code" , "shippingInfo_shipmentDetails_address_fullName_firstName" , "shippingInfo_shipmentDetails_priceData_price", "shippingInfo_shipmentDetails_priceData_taxIncludedInPrice" , "shippingInfo_shipmentDetails_tax", "shippingInfo_shipmentDetails_discount" , "shippingInfo_shipmentDetails_address_street_name" , "shippingInfo_shipmentDetails_address_street_number" , "shippingInfo_shipmentDetails_address_email" , "shippingInfo_shipmentDetails_address_phone" , "shippingInfo_shipmentDetails_address_zipCode" , "shippingInfo_shipmentDetails_address_city" , "shippingInfo_shipmentDetails_address_subdivision" , "shippingInfo_shipmentDetails_address_country" , "shippingInfo_shipmentDetails_address_fullName_lastName" , "shippingInfo_deliveryOption", "buyerInfo_email", "billingInfo_address_email"]
wix_orders.drop(columns=drop_list, inplace=True)

In [ ]:
wix_orders.reset_index(drop=True, inplace=True)
line_items.reset_index(drop=True, inplace=True)

In [ ]:
wix_orders[730:]

In [ ]:
line_items.to_csv("data/line_items.csv")

In [ ]:
wix_orders.to_csv("data/wix_orders.csv")

In [ ]:
# normalize column lineitems und joinen
lineItems = pd.json_normalize(data, record_path='lineItems', sep="_")
all_data = pd.concat([wix_orders,lineItems], axis=1)
all_data.drop(columns="lineItems", inplace=True)

In [ ]:
all_data.to_csv("data/all_data.csv")

In [ ]:
#column activities normalisiert, kann auch geadded werden, sprengt jedoch die Datenreihen
activities_df = pd.json_normalize(data, record_path='activities')
activities_df.to_csv("data/activities_df.csv")